In [ ]:
import os
LANGCHAIN_API_KEY = os.getenv(key="LANGCHAIN_API_KEY")
LANGCHAIN_ENDPOINT = os.getenv(key="LANGCHAIN_ENDPOINT")
LANGCHAIN_TRACING_V2 = os.getenv(key="LANGCHAIN_TRACING_V2")

In [ ]:
from langchain.chat_models.ollama import ChatOllama
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
import requests
from bs4 import BeautifulSoup

In [ ]:
template = """Summarize the following Question based on the context:

Question: {question}

Context:

{context}"""

prompt = ChatPromptTemplate.from_template(template)

In [ ]:
prompt

In [ ]:

llm = ChatOllama(model="phi", temperature=0, timeout=300)

In [ ]:
url = "https://blog.langchain.dev/announcing-langsmith"

In [ ]:
def scrape_text(url):
    try:
        response = requests.get(url)

        if response.status_code == 200:
            soup = BeautifulSoup(response.text, "html.parser")

            page_text = soup.get_text(separator=" ", strip=True)

            return page_text
        else:
            return f"Failed to retrieve the webpage: Status code {response.status_code}"
    except Exception as e:
        print(e)
        return f"Failed to retrieve the webpage: {e}"

In [ ]:
scrap = scrape_text(url)

In [ ]:
scrap[:10000]

In [ ]:
import asyncio
import sys

if sys.platform:
    asyncio.set_event_loop_policy(asyncio.WindowsSelectorEventLoopPolicy())

In [ ]:
sys.platform

In [1]:
import os
import asyncio
import sys
import requests
from bs4 import BeautifulSoup
from langchain.chat_models.ollama import ChatOllama
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper
import json
from langchain_core.output_parsers import JsonOutputParser

In [2]:
# Set environment variables
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")
LANGCHAIN_ENDPOINT = os.getenv("LANGCHAIN_ENDPOINT")
LANGCHAIN_TRACING_V2 = os.getenv("LANGCHAIN_TRACING_V2")

In [ ]:
# Configure asyncio for Windows
if sys.platform:
    asyncio.set_event_loop_policy(asyncio.WindowsSelectorEventLoopPolicy())

In [3]:
# DuckDuckGo Search
RESULTS_PER_QUESTION = 3
ddg_search = DuckDuckGoSearchAPIWrapper()


In [4]:
# Function to perform web search
def web_search(query: str, num_results: int = RESULTS_PER_QUESTION):
    results = ddg_search.results(query, num_results)
    return [r["link"] for r in results]

In [6]:
# Prompt template for summarization
SUMMARY_TEMPLATE = """{text} 

-----------
Using the above text, answer in short the following question: 
> {question}
-----------
if the question cannot be answered using the text, imply summarize the text. Include all factual information, numbers, stats etc if available."""
SUMMARY_PROMPT = ChatPromptTemplate.from_template(SUMMARY_TEMPLATE)

In [7]:
# ChatOllama model for summarization
llm = ChatOllama(model="phi", temperature=0, timeout=300)

In [8]:
# Function to scrape text from a webpage
def scrape_text(url: str):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, "html.parser")
            page_text = soup.get_text(separator=" ", strip=True)
            return page_text
        else:
            return f"Failed to retrieve the webpage: Status code {response.status_code}"
    except Exception as e:
        print(e)
        return f"Failed to retrieve the webpage: {e}"

In [11]:
# Runnable chain for scraping and summarizing text
scrape_and_summarize_chain = RunnablePassthrough.assign(
    text=lambda x: scrape_text(x["url"])[:10000]
) | SUMMARY_PROMPT | llm | StrOutputParser()

In [12]:
url = "https://blog.langchain.dev/announcing-langsmith"
url

'https://blog.langchain.dev/announcing-langsmith'

In [13]:
scrape_and_summarize_chain.invoke(
    {
        "question": "what is langsmith?",
        "url": url
    }
)

' LangSmith is a tool that helps developers debug and evaluate their language models by providing an easy-to-use interface to trace and visualize the sequence of calls in a chain or prompt. It also allows for testing and monitoring the performance of the application. LangSmith can be used to improve the efficiency and effectiveness of LLM applications, and is particularly useful for teams that are striving for a more effective approach.\n'

In [14]:
# Runnable chain for web search
web_search_chain = RunnablePassthrough.assign(
    urls=lambda x: web_search(x["question"])
) | (lambda x: [{"question": x["question"], "url": u} for u in x["urls"]]) | scrape_and_summarize_chain.map()


In [15]:
web_search_chain.invoke(
    {
        "question": "what is langsmith?"
    }
)

d:\Gen_AI_Tutorials\langchain\venv\lib\site-packages\curl_cffi\aio.py:204: UserWarning: Curlm alread closed! quitting from process_data
  warnings.warn("Curlm alread closed! quitting from process_data")


[' LangSmith is a tool that helps developers debug and evaluate their language models by providing an easy-to-use interface for tracing and evaluating complex agent prompt chains. It allows users to create datasets of examples they care about and run any changed prompts over those data sets. LangSmith also provides tools for monitoring the performance of applications, including system-level performance, model/chain performance, and user interactions.\n',
 ' LangSmith is a tool that allows users to evaluate their language models and chatbots by providing them with prompts and evaluating their responses. It provides a platform for users to test their models in real-time and get feedback on their performance. LangSmith also offers tools for analyzing the output of the model, such as parsing and sentiment analysis.\n',
 ' LangSmith is a tool that helps developers build and deploy their own language models for various applications. It provides an extensible platform with features like code 

In [16]:
# Prompt template for search queries
SEARCH_PROMPT = ChatPromptTemplate.from_messages(
    [
        (
            "user",
            "Write 3 Google search queries to search online that form an objective opinion from the following: {question}\n"
            "You must respond with a list of strings in the following format: ['query 1', 'query 2', 'query 3'].",
        ),
    ]
)

In [17]:
# ChatOllama model for handling search prompts
gemma_llm = ChatOllama(model="gemma:2b", temperature=0, timeout=300)

In [18]:
# Runnable chain for processing search prompts
search_question_chain = SEARCH_PROMPT | gemma_llm | StrOutputParser()


In [19]:
# Invocation of the search_question_chain
res = search_question_chain.invoke(
    {
        "question": "what is the difference between langchain and langsmith?"
    }
)

In [20]:
print(res)

1. Langchain vs Langsmith: Understanding the Differences
2. Exploring the Landscape of Langchains and Langsmiths
3. Insights into Langchain and Langsmith: Key Differences and Similarities


In [21]:
type(res)

str

In [24]:
queries = res.splitlines()

In [25]:
print(len(queries))
print(queries)
print(type(queries))


3
['1. Langchain vs Langsmith: Understanding the Differences', '2. Exploring the Landscape of Langchains and Langsmiths', '3. Insights into Langchain and Langsmith: Key Differences and Similarities']
<class 'list'>


In [50]:
print(queries[1])

2. Exploring the Landscape of Langchains and Langsmiths


In [55]:
output_function = lambda x: [{"question": q} for q in x]
output = output_function(queries)

In [56]:
print(output)

[{'question': '1. Langchain vs Langsmith: Understanding the Differences'}, {'question': '2. Exploring the Landscape of Langchains and Langsmiths'}, {'question': '3. Insights into Langchain and Langsmith: A Comparative Analysis'}]


In [31]:
from langchain_core.output_parsers import StrOutputParser

# Define your search_question_chain
search_question_chain1 = SEARCH_PROMPT | gemma_llm | StrOutputParser() | (lambda x: x.splitlines())


In [32]:
# Invocation of the search_question_chain
res1 = search_question_chain1.invoke(
    {
        "question": "what is the difference between langchain and langsmith?"
    }
)

In [33]:
print(res1)

['1. Langchain vs Langsmith: Understanding the Differences', '2. Exploring the Landscape of Langchains and Langsmiths', '3. Insights into Langchain and Langsmith: Key Differences and Similarities']


In [68]:
search_question_chain2 = (
    SEARCH_PROMPT
    | gemma_llm
    | StrOutputParser()
    | (lambda x: [{"question": q} for q in x.splitlines()])
)

In [69]:
res2 = search_question_chain2.invoke(
    {
        "question": "what is the difference between langchain and langsmith?"
    }
)

In [70]:
print(res2)

[{'question': '1. Langchain vs Langsmith: Understanding the Differences'}, {'question': '2. Exploring the Landscape of Langchains and Langsmiths'}, {'question': '3. Insights into Langchain and Langsmith: A Comparative Analysis'}]


In [61]:
print(res1)

['1. Langchain vs Langsmith: Understanding the Differences', '2. Exploring the Landscape of Langchains and Langsmiths', '3. Insights into Langchain and Langsmith: A Comparative Analysis']


In [37]:
output_function1 = lambda x: [{"question": q} for q in x]
output = output_function1(res1)

In [38]:
print(output)

[{'question': '1. Langchain vs Langsmith: Understanding the Differences'}, {'question': '2. Exploring the Landscape of Langchains and Langsmiths'}, {'question': '3. Insights into Langchain and Langsmith: Key Differences and Similarities'}]


In [55]:
# chain = search_question_chain1 | (lambda x: [{"question": q} for q in x]) | web_search_chain.map()
chain = RunnablePassthrough.assign(
    queries = search_question_chain1
) | (lambda x: [{"question": q} for q in x])

In [ ]:
# Runnable chain for web search
web_search_chain = RunnablePassthrough.assign(
    urls=lambda x: web_search(x["question"])
) | (lambda x: [{"question": x["question"], "url": u} for u in x["urls"]]) | scrape_and_summarize_chain.map()


In [56]:
chain.invoke(
    {
        "question": "what is the difference between langchain and huggingface?"
    }
)

[{'question': 'question'}, {'question': 'queries'}]